<a href="https://colab.research.google.com/github/siliconemonster/IA-2020.1/blob/main/Algoritmo_Genetico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import random

In [10]:
def geraTabuleiro(n):
  tabuleiro = np.zeros(( n), dtype=int)
  #print(tabuleiro)

  for x in range(n):
    posicaoRainha = random.randint(1, n)
    #print(posicaoRainha)
    tabuleiro[x] = posicaoRainha

  #print("Tabuleiro Atual:")
  #print(tabuleiro)
  return tabuleiro

geraTabuleiro(4)

array([3, 3, 1, 1])

In [28]:
def avaliaAtaques(tabuleiro):
  n = len(tabuleiro)
  ataques = 0

  for i in range(n-1): # caso esteja na mesma coluna
    for j in range(i+1, n):
      if (tabuleiro[i] == tabuleiro[j]):
        ataques = ataques + 1

  for i in range(n-1): # caso esteja na mesma diagonal. Para baixo e para cima, respectivamente
    for j in range(i+1, n):
      if ((tabuleiro[i] == tabuleiro[j] + (j - i)) or (tabuleiro[i] == tabuleiro[j] - (j - i))):
        ataques = ataques + 1

  #print("Número de ataques:")
  #print(ataques)
  return ataques

tabuleiro = geraTabuleiro(4)
avaliaAtaques(tabuleiro)

3

In [26]:
def geraPopulacao(numIndividuos, n):
  populacao = np.zeros((numIndividuos, n), dtype=int)

  for i in range(numIndividuos):
    populacao[i] = geraTabuleiro(n) ##### FALTA LIDAR COM ELEMENTOS REPETIDOS

  return populacao

geraPopulacao(10, 4)

array([[1, 1, 4, 3],
       [1, 2, 4, 1],
       [1, 1, 4, 4],
       [3, 1, 1, 3],
       [2, 2, 4, 3],
       [1, 3, 1, 3],
       [3, 3, 3, 1],
       [2, 4, 3, 2],
       [4, 4, 4, 4],
       [2, 4, 3, 2]])

In [15]:
def reproduzir(pai1, pai2):
  filhos = np.zeros(2, dtype=int)
  n = len(pai1)
  c = random.randint(1, n)

  filhos = # QUE

  return filhos

In [30]:
def mutacao(elemento):  #### EXISTEM OUTROS TIPOS DE MUTAÇÃO
  n = len(elemento)
  aleatorio = random.randint(0, n)

  if (aleatorio == n):
    aux = elemento[aleatorio]
    elemento[aleatorio] = elemento[0]
    elemento[0] = aux

  else:
    aux = elemento[aleatorio]
    elemento[aleatorio] = elemento[aleatorio + 1]
    elemento[aleatorio + 1] = aux

  return elemento

#mutacao([1,2,3,4,5,6,7,8])

[1, 3, 2, 4, 5, 6, 7, 8]

O algoritmo genético:

In [ ]:
def genetico(populacao, probabilidade):

  novaPopulacao = []

  while True:
    for i in range(len(populacao)):
      aux1 = random.randint(0, (len(populacao)))
      aux2 = random.randint(0, (len(populacao)))
      pai1 = populacao[aux1]
      pai2 = populacao[aux2] ##### FALTA GARANTIR QUE PAIS NÃO SÃO REPETIDOS
      filho = reproduzir(pai1, pai2)
      probCalculada = random.uniform(0, probabilidade)
      if (probCalculada <= probabilidade):
        filho = mutacao(filho)
      novaPopulacao[i] = filho
    populacao = novaPopulacao # acho que isso está errado
    if (condicoesDeParada): # determinar condições de parada
      return melhorElemento # determinar quem é o melhor elemento

populacao = geraPopulacao(10, 4)
genetico(populacao)